In [1]:
import warnings
warnings.filterwarnings('ignore')
import requests
import folium
from pandas.io.json import json_normalize

In [2]:
# 전국 또는 특정 지역의 스타벅스 매장 위치를 찾아 지도위에 표시하기
# 1. requests 모듈로 스타벅스 매장 위치 데이터를 가져와서 파이썬에서 처리하기 위해 딕셔너리 타입으로 변환한다.
# 2. json_normalize() 함수로 json이 딕셔너리로 변환된 데이터를 판다스 데이터프레임으로 변환한다.
# 3. foliumm 모듈을 사용해서 지도를 표시하고 지도위의 스타벅스 매장 위치에 마커를 표시한다.


In [3]:
# 서울 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=X1U9WNFPZU'
request = requests.post(targetSite, data = {
    'ins_lat': '37.3843161', #위도
    'ins_lng': '127.0722198', #경도
    'p_sido_cd': '01', #시도코드     #강남구만 보고싶으면 'p_sido_cd': ''로하도 되고 강남 구군코드 'p_gugun_cd': '0101'만 입력해도된다
    'p_gugun_cd': '',  #구군코드
    'in_biz_cd' : '',  #?
    'set_date' : '',   #?
    'iend' : '1700',    #'iend': '100', #서버에서 응답하는 최대 매장 개수. 100개보다 더 많은 결과를 보고싶으면 더 큰 숫자로 지정하면 된다.
    #'rndCod': 'BKMWNBSE4A' 주소 끝 r= X1U9WNFPZU이 rndCod를 의미해서 쓸필요없다
})
storelist = request.json()


In [4]:
#json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
#json_normalize(데이터프레임으로 변환할 데이터가 저장된 딕셔너리, 딕셔너리의 key 중에서 데이터프레임으로 변환할 key)
star_df = json_normalize(storelist, 'list')
print(type(storelist))

<class 'dict'>


In [5]:
star_df.head()

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,t22,t21,p90,t05,t30,t36,t27,t29,t43,t48
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [6]:
print(storelist['list'][0])

{'seq': 0, 'sido_cd': None, 'sido_nm': None, 'gugun_cd': None, 'gugun_nm': None, 'code_order': None, 'view_yn': None, 'store_num': None, 'sido': None, 'gugun': None, 'address': None, 'new_img_nm': None, 'p_pro_seq': 0, 'p_view_yn': None, 'p_sido_cd': '', 'p_gugun_cd': '', 'p_store_nm': None, 'p_theme_cd': None, 'p_wireless_yn': None, 'p_smoking_yn': None, 'p_book_yn': None, 'p_music_yn': None, 'p_terrace_yn': None, 'p_table_yn': None, 'p_takeout_yn': None, 'p_parking_yn': None, 'p_dollar_assent': None, 'p_card_recharge': None, 'p_subway_yn': None, 'stb_store_file_renew': None, 'stb_store_theme_renew': None, 'stb_store_time_renew': None, 'stb_store_lsm': None, 's_code': '1509', 's_name': '역삼아레나빌딩', 'tel': '1522-3232', 'fax': '02-568-3763', 'sido_code': '01', 'sido_name': '서울', 'gugun_code': '0101', 'gugun_name': '강남구', 'addr': '서울특별시 강남구 역삼동 721-13 아레나빌딩', 'park_info': None, 'new_state': None, 'theme_state': 'T05@T08@T16@T17@T20@T21@T30@T36@T52@P80@P90', 'new_bool': 0, 'search_text': ''

In [7]:
# 작업에 필요한 컬럼 몇 개를 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터프레임을 만든다.
# s_name        => 지점이르 
# sido_code     => 시도 코드
# sido_name     => 시도 이름
# gugun_code    => 구군 코드 
# gugun_name    => 구군 이름
# doro_address => 도로명 주소
# lat           => 위도
# lot           => 경도

# 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 컬럼 개수가 1개라면 컬럼 이름만 적으면 되지만 얻어올 컬럼이 2개
# 이상이라면 얻어올 컬럼 이름을 리스트로 만들어서 얻어와야 한다.

star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address','lat', 'lot']]

In [8]:
star_df_map.head()

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,역삼아레나빌딩,01,서울,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,01,서울,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,01,서울,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.514132,127.020563
3,국기원사거리,01,서울,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,스탈릿대치R,01,서울,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583


In [9]:
star_df_map.dtypes

s_name          object
sido_code       object
sido_name       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [10]:
# astype() 함수로 위도, 경도 데이터의 타입을 object(문자열)에서 float(실수)로 변경한다.
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)
star_df_map.dtypes

s_name           object
sido_code        object
sido_name        object
gugun_code       object
gugun_name       object
doro_address     object
lat             float64
lot             float64
dtype: object

In [11]:
# 종로관철점
star_df_map[star_df_map['s_name'] == '종로관철']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
419,종로관철,01,서울,0118,종로구,"서울특별시 종로구 종로12길 21, 2층 (관철동)",37.569058,126.986013


In [12]:
seoul_star_map = folium.Map(location = [star_df_map['lat'].mean(), star_df_map['lot'].mean()], zoom_start= 12)

for index, data in star_df_map.iterrows():
    # print(index, data)
    store_info = data['s_name'] + '점, 주소: ' + data['doro_address']
    popup = folium.Popup(store_info, max_width = 250)
    folium.Marker(location = [data['lat'], data['lot']], popup = popup).add_to(seoul_star_map)
    
seoul_star_map.save('./Output/seoul_star_map')
seoul_star_map

In [13]:
# 강남 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=X1U9WNFPZU'
request = requests.post(targetSite, data = {
    'ins_lat': '37.3843161', #위도
    'ins_lng': '127.0722198', #경도
    'p_sido_cd': '', #시도코드     #강남구만 보고싶으면 'p_sido_cd': ''로하도 되고 강남 구군코드 'p_gugun_cd': '0101'만 입력해도된다
    'p_gugun_cd': '0101',  #구군코드
    'in_biz_cd' : '',  #?
    'set_date' : '',   #?
    'iend' : '1700',    #'iend': '100', #서버에서 응답하는 최대 매장 개수. 100개보다 더 많은 결과를 보고싶으면 더 큰 숫자로 지정하면 된다.
    #'rndCod': 'BKMWNBSE4A' 주소 끝 r= X1U9WNFPZU이 rndCod를 의미해서 쓸필요없다
})

gangnam_storelist = request.json()

gangnam_star_df = json_normalize(gangnam_storelist, 'list')

gangnam_star_df_map = gangnam_star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address','lat', 'lot']]

gangnam_star_df_map['lat'] = gangnam_star_df_map['lat'].astype(float)
gangnam_star_df_map['lot'] = gangnam_star_df_map['lot'].astype(float)

gangnam_star_map = folium.Map(location = [gangnam_star_df_map['lat'].mean(), gangnam_star_df_map['lot'].mean()], zoom_start= 12)

for index, data in gangnam_star_df_map.iterrows():
    # print(index, data)
    store_info = data['s_name'] + '점, 주소: ' + data['doro_address']
    popup = folium.Popup(store_info, max_width = 250)
    folium.Marker(location = [data['lat'], data['lot']], popup = popup).add_to(gangnam_star_map)
    
gangnam_star_map.save('./Output/gangnam_star_map')
gangnam_star_map

In [14]:
# 경기도 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=X1U9WNFPZU'
request = requests.post(targetSite, data = {
    'ins_lat': '37.3843161', #위도
    'ins_lng': '127.0722198', #경도
    'p_sido_cd': '08', #시도코드     #강남구만 보고싶으면 'p_sido_cd': ''로하도 되고 강남 구군코드 'p_gugun_cd': '0101'만 입력해도된다
    'p_gugun_cd': '',  #구군코드
    'in_biz_cd' : '',  #?
    'set_date' : '',   #?
    'iend' : '1700',    #'iend': '100', #서버에서 응답하는 최대 매장 개수. 100개보다 더 많은 결과를 보고싶으면 더 큰 숫자로 지정하면 된다.
    #'rndCod': 'BKMWNBSE4A' 주소 끝 r= X1U9WNFPZU이 rndCod를 의미해서 쓸필요없다
})

gyongi_storelist = request.json()

gyongi_star_df = json_normalize(gyongi_storelist, 'list')

gyongi_star_df_map = gyongi_star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address','lat', 'lot']]

gyongi_star_df_map['lat'] = gyongi_star_df_map['lat'].astype(float)
gyongi_star_df_map['lot'] = gyongi_star_df_map['lot'].astype(float)

gyongi_star_map = folium.Map(location = [gyongi_star_df_map['lat'].mean(), gyongi_star_df_map['lot'].mean()], zoom_start= 12)

for index, data in gyongi_star_df_map.iterrows():
    # print(index, data)
    store_info = data['s_name'] + '점, 주소: ' + data['doro_address']
    popup = folium.Popup(store_info, max_width = 250)
    folium.Marker(location = [data['lat'], data['lot']], popup = popup).add_to(gyongi_star_map)
    
gyongi_star_map.save('./Output/gyongi_star_map')
gyongi_star_map

In [15]:
# 전국 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=X1U9WNFPZU'
request = requests.post(targetSite, data = {
    'ins_lat': '37.3843161', #위도
    'ins_lng': '127.0722198', #경도
    'p_sido_cd': '', #시도코드     #강남구만 보고싶으면 'p_sido_cd': ''로하도 되고 강남 구군코드 'p_gugun_cd': '0101'만 입력해도된다
    'p_gugun_cd': '',  #구군코드
    'in_biz_cd' : '',  #?
    'set_date' : '',   #?
    'iend' : '1700',    #'iend': '100', #서버에서 응답하는 최대 매장 개수. 100개보다 더 많은 결과를 보고싶으면 더 큰 숫자로 지정하면 된다.
    #'rndCod': 'BKMWNBSE4A' 주소 끝 r= X1U9WNFPZU이 rndCod를 의미해서 쓸필요없다
})

korea_storelist = request.json()

korea_star_df = json_normalize(korea_storelist, 'list')

korea_star_df_map = korea_star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address','lat', 'lot']]

korea_star_df_map['lat'] = korea_star_df_map['lat'].astype(float)
korea_star_df_map['lot'] = korea_star_df_map['lot'].astype(float)

korea_star_map = folium.Map(location = [korea_star_df_map['lat'].mean(), korea_star_df_map['lot'].mean()], zoom_start= 8)

for index, data in korea_star_df_map.iterrows():
    # print(index, data)
    store_info = data['s_name'] + '점, 주소: ' + data['doro_address']
    popup = folium.Popup(store_info, max_width = 250)
    folium.Marker(location = [data['lat'], data['lot']], popup = popup).add_to(korea_star_map)
    
korea_star_map.save('./Output/korea_star_map')
korea_star_map